In [2]:
import pandas as pd
from hazm import *
import re
import numpy as np

In [3]:
Data = pd.read_csv('Instagram labeled comments.csv')

In [4]:
X_train, Y_train = Data['comment'], Data['sentiment']

In [5]:
normalizer = Normalizer()

In [6]:
Y_train = np.array([Y_train[i] for i in range(len(X_train)) if not re.search(r'[a/z]|[A/Z]', X_train[i])])

In [7]:
X_train = np.array([X_train[i] for i in range(len(X_train)) if not re.search(r'[a/z]|[A/Z]', X_train[i])])

In [8]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", flags=re.UNICODE)

X_train_nonemoji = [emoji_pattern.sub(r'', text) for text in X_train]

In [9]:
X_train_nonemoji[0]

'آقای رشیدپور من تا هفته NUM۲ بچه\u200cام پسر بود و لحظه به دنیا اومدنش دختر شد خداروشکررر . شما ببین این وروجک چجوری من و باباش و دکتر رو سر انگشتش چرخوند'

In [10]:
X_train_nonemoji_normalized = [normalizer.normalize(i) for i in X_train_nonemoji]

In [11]:
X_train_nonemoji_normalized[0]

'آقای رشیدپور من تا هفته NUM۲ بچه\u200cام پسر بود و لحظه به دنیا اومدنش دختر شد خداروشکررر. شما ببین این وروجک چجوری من و باباش و دکتر رو سر انگشتش چرخوند'

In [12]:
tagger = POSTagger(model='resources-0/postagger.model')

In [13]:
sent_tokenize(X_train_nonemoji_normalized[0])

['آقای رشیدپور من تا هفته NUM۲ بچه\u200cام پسر بود و لحظه به دنیا اومدنش دختر شد خداروشکررر.',
 'شما ببین این وروجک چجوری من و باباش و دکتر رو سر انگشتش چرخوند']

In [14]:
final = np.hstack((np.array(X_train_nonemoji_normalized).reshape(8335,1),np.array(Y_train).reshape(8335,1)))

In [15]:
Data_final = pd.DataFrame(final)

In [16]:
Data_final.rename(columns = { 1 :'sentiment', 0 :'comment' }, inplace = True)

In [17]:
Data_final[Data_final['comment'].str.strip().astype(bool)]

,comment,sentiment
0,آقای رشیدپور من تا هفته NUM۲ بچه‌ام پسر بود و ...,1
1,چقدر این دزد باحال بود,1
2,سلام اگر ممکنه از کلمه سی تی اسکن تو برنامتون ...,-1
3,شما پول اینترنتت و گوشیت رو هزینه بچت کن,0
4,آقای رشید پور به روستای ما بن گوشت یخ زده دادن...,-1
...,...,...
8330,این ضایعه را به ملت بزرگ ایران به ویژه خانواده...,-1
8331,شبیه افغانی هاس,0
8332,باسلام تکلیف کشتی که بانفتکش ما تصادف کرد چی ش...,0
8333,گالری اینترنتی فرشاد,0


In [18]:
Data_final.to_csv('pre-proc-data.csv')